In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/capstone/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/capstone





**install spacy**




In [ ]:
# 
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [3]:
import spacy
spacy.load('en_core_web_trf')

**install OpenNRE**

In [ ]:
!git clone https://github.com/thunlp/OpenNRE.git

In [8]:
%cd OpenNRE

/content/drive/MyDrive/capstone/OpenNRE


In [ ]:
pip install -r requirements.txt

In [ ]:
!python setup.py install

In [11]:
%cd ../

/content/drive/MyDrive/capstone


**parse pubMed data**

In [ ]:
!pip install pubmed-parser

In [4]:
import pubmed_parser as pp
path_xml = pp.list_xml_path('data') # list all xml paths under directory


In [5]:
pubmed_dict = pp.parse_medline_xml(path_xml[0]) 

In [10]:
print(pubmed_dict[0]['abstract'])

Severe COVID-19 is associated with venous thromboembolic events and and immuno-thrombotic phenomena, responsible for pulmonary vascular damage. This review summarizes the current knowledge on thrombotic risk in COVID-19 inpatients, the potential predictive factors (including D-dimer) and the randomized trials studying the effect of intermediate or therapeutic-dose anticoagulation on the clinical and thrombotic prognosis. Despite the initial hope, therapeutic anticoagulation does not improve the clinical prognosis in critically ill inpatients, and standard prophylactic anticoagulation is therefore recommended. In non-critical inpatients, the use of therapeutic anticoagulation may help reduce the risk of severe clinical deterioration, but its risk-benefit will be clarified in ongoing studies and meta-analyzes.


**auto-summarization transformers**

In [6]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = pubmed_dict[0]['abstract']

In [ ]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)